In [44]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
import random
import math




In [45]:
def dataread():
  ltrs = pd.read_csv("letter-recognition.csv")
  return ltrs

def accu_poly_ker(X_train, Y_train,X_test,Y_test):
  non_linear_model = SVC(kernel='poly')
  non_linear_model.fit(X_train, Y_train)
  y_pred = non_linear_model.predict(X_test)
  print("Accuracy polynomial_ker:", metrics.accuracy_score(y_true=Y_test, y_pred=y_pred), "\n")

def accurbf_ker(X_train, Y_train,X_test,Y_test):
  non_linear_model = SVC(kernel='rbf')
  non_linear_model.fit(X_train, Y_train)
  y_pred = non_linear_model.predict(X_test)
  print("Accuracy rbf_ker:", metrics.accuracy_score(y_true=Y_test, y_pred=y_pred), "\n")

def acculinear_ker(X_train, Y_train,X_test,Y_test):
  model_linear = SVC(kernel='linear')
  model_linear.fit(X_train, Y_train)
  y_pred = model_linear.predict(X_test)
  print("Accuracy linear_ker : ", metrics.accuracy_score(y_true=Y_test, y_pred=y_pred), "\n")


def k_times_markov(X_train, Y_train, k , q = 1.2):
    samp = []
    for zxx in range(k):
        
        In_samp = np.concatenate((np.vstack(X_train), np.vstack(Y_train.to_numpy())), axis = 1)
        DR = random.sample(list(In_samp), int(2000)) 
        l = len(DR)
        mneg = 0
        mplus = 0
        index = np.random.choice(len(DR), 1, replace=False)  
        DRa = np.array(DR)
        zx = DRa[index][0]
        if l%2 == 0:
            if zx[16] == 'A':
                mplus += 1;
            else:
                mneg += 1
        model_linear = SVC(kernel='linear')
        model_linear.fit(X_train, Y_train)
        
        yzx = model_linear.predict(np.array(zx)[0:16].reshape(1, -1))
        xyz1 = 1
        if not yzx==zx[16]:
            xyz1 = 2 
        lzx = math.exp(0-xyz1)

        Pd = 0
        Pd1 = 0
        
        while(mplus + mneg < l ):
            zstar = DRa[np.random.choice(len(DR), 1, replace=False)][0]
            ystar = model_linear.predict(np.array(zstar)[ 0:16].reshape(1, -1)) #
            xyz = 1
            if not ystar==zstar[16]:
                xyz = 2 
            lzstar = math.exp(0-xyz)
            P = math.exp(lzx-lzstar)
            yt = zx[16]
            zx = zstar

            if P == 1:
                if zx[16] == yt:
                    r = random.uniform(0.001, 1.0)
                    if r <= P:
                        samp.append(zstar) 
                else:
                    ct = 1 if yt=='A' else -1
                    cst = 1 if ystar=='A' else -1
                    Pd = math.exp(ct*xyz1-cst*xyz)
                    r = random.uniform(0.001, 1.0)
                    if r <= Pd:
                        samp.append(zstar) 

            if len(samp) == k:
                Pd1 = q*P
                samp.append(zstar)
                
            zxp1 = zstar
            if yt == 'A':
                mplus += 1
            else:
                mneg += 1

            if P > 1 or Pd > 1 or Pd1 > 1:
                samp.append(zstar)
    return samp

def calculateaccuracy(X_train, y_train,X_test):
  print("Enter the value of k")
  k = int(input())
  nsamp = np.array(k_times_markov(X_train, y_train,k))
  print("size_of_nsamp")
  print(nsamp.shape,"\n")
  X_train = nsamp[:, 0:16]
  y_train = nsamp[:, 16]
  print("size_of_X_train")
  print(X_train.shape,"\n")
  print("size_of_Y_train")
  print(y_train.shape,"\n")
  Y_train = []
  for i in y_train:
      Y_train.append(ord(i))
  Y_test = []
  for i in y_test:
     Y_test.append(ord(i))
  acculinear_ker(X_train, Y_train,X_test,Y_test)
  accurbf_ker(X_train, Y_train,X_test,Y_test)
  accu_poly_ker(X_train, Y_train,X_test,Y_test)


In [46]:
ltrs=dataread()
print("size_of_dataset ", ltrs.shape, "\n")
ltrs.columns = ['letter', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar', 'ybar', 'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge', 'xedgey', 'yedge', 'yedgex']
print(ltrs.columns)
order = list(np.sort(ltrs['letter'].unique()))
round(ltrs.drop('letter', axis=1).mean(), 2)
X = ltrs.drop("letter", axis = 1)
y = ltrs['letter']
X_scaled = scale(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 101)
calculateaccuracy(X_train, y_train,X_test)

size_of_dataset  (20000, 17) 

Index(['letter', 'xbox', 'ybox', 'width', 'height', 'onpix', 'xbar', 'ybar',
       'x2bar', 'y2bar', 'xybar', 'x2ybar', 'xy2bar', 'xedge', 'xedgey',
       'yedge', 'yedgex'],
      dtype='object')
Enter the value of k
10
size_of_nsamp
(16709, 17) 

size_of_X_train
(16709, 16) 

size_of_Y_train
(16709,) 

Accuracy linear_ker :  0.8413333333333334 

Accuracy rbf_ker: 0.9348333333333333 

Accuracy polynomial_ker: 0.8766666666666667 

